# 환경설정

In [1]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive

gauth = GoogleAuth()
gauth.LocalWebserverAuth()  # 처음 실행 시 브라우저 인증
drive = GoogleDrive(gauth)

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?client_id=1071344378062-kv36e6mgl1vnr89evq958br79d063h8p.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&access_type=offline&response_type=code

Authentication successful.


In [2]:
from dotenv import load_dotenv
import os

load_dotenv()
google_dirve_md_id = os.getenv("GOOGLE_DRIVE_MD_ID")

## Load Documents

In [3]:
# .md 파일만 검색
file_list = drive.ListFile({
    'q': f"'{google_dirve_md_id}' in parents and title contains '.md' and trashed=false"
}).GetList()
len(file_list)

43

In [4]:
from langchain_core.documents import Document

# 자막 데이터 로드
docs = []
for file in file_list:
    source = file['title']
    content = file.GetContentString()
    doc = Document(
        page_content=content, 
        metadata={"source": source}
    )
    docs.append(doc)

    print(f'{source} : {len(content)}')
    print('---'*10)

[즉문즉설] 제83회 신랑이 바람을 피웠어요..ko.md : 639
------------------------------
(EN/FR/中文 SUB) 제314회 전생과 윤회에 대하여.ko.md : 1197
------------------------------
[즉문즉설]딸이 정토회에 너무 빠져있어서 화가 날 정도예요.ko.md : 1035
------------------------------
버티는 대통령, 어떻게 이해해요?.ko.md : 1851
------------------------------
(ENG /中文 SUB)  법륜스님 즉문즉설 1203회 행복은 마음먹기에 따라 달라진다.ko.md : 1592
------------------------------
부부가 떨어져 사는 것이 옳은지요?.ko.md : 1522
------------------------------
제336회 부익부 빈익빈의 현상.ko.md : 862
------------------------------
남편이 짠돌이.ko.md : 553
------------------------------
결혼을 못했습니다.ko.md : 1374
------------------------------
장인에게 혼난 후 아내까지 미워요.ko.md : 786
------------------------------
(Eng/中文-SUB) 즉문즉설1026회 여자친구 몰래 바람을 피웠는데 돌아가야할지 고민이에요.ko.md : 591
------------------------------
제553회 헤어진 남친을 다시 만나고 있는데, 그만 헤어져야 할 지 만나야 할 지.ko.md : 3031
------------------------------
[즉문즉설]이혼 도장은 찍었지만 마음이 답답합니다.ko.md : 1005
------------------------------
제513회 남을 바꾸려고 하지마라(법문).ko.md : 565
------------------------------
(EN/中

## Indexing

In [5]:
from langchain_openai.embeddings import OpenAIEmbeddings

# 임베딩 정의
embeddings = OpenAIEmbeddings(
    model="text-embedding-3-small",
    dimensions=1024
)

# 문서 임베딩
# document_embeddings = embeddings.embed_documents(
#     [chunk.page_content for chunk in docs]
# )

In [9]:
from langchain.vectorstores import Chroma

# 벡터 스토어 저장
vectorstore = Chroma.from_documents(
    documents=docs,
    embedding=embeddings,
    collection_name="content-250618",
    persist_directory="./chroma_db"
)

In [14]:
# 검색 테스트 
query = "불안해요."

retriever_mmr = vectorstore.as_retriever(
    search_type='mmr',
    search_kwargs={
        'k': 3,                 # 검색할 문서의 수
        'fetch_k': 8,           # mmr 알고리즘에 전달할 문서의 수 (fetch_k > k)
        'lambda_mult': 0.3,     # 다양성을 고려하는 정도 (1은 최소 다양성, 0은 최대 다양성을 의미. 기본값은 0.5)
        },
)
retrieved_docs = retriever_mmr.invoke(query)

print(f"쿼리: {query}")
print("검색 결과:")
for doc in retrieved_docs:
    print(doc.metadata['source'])
    print('---'*20)
    print(doc.page_content[:100])
    print('==='*20)


쿼리: 불안해요.
검색 결과:
제553회 헤어진 남친을 다시 만나고 있는데, 그만 헤어져야 할 지 만나야 할 지.ko.md
------------------------------------------------------------
질문자: 10년 된 남친이 있었는데, 2년 전 헤어지고 다시 만나요. 집안 환경 때문에 남친 부모님이 안 좋아하셔서 헤어졌는데, 저희 집이 매일 싸우는 환경이라 저도 행복하지 못했
제336회 부익부 빈익빈의 현상.ko.md
------------------------------------------------------------
질문자: 스님, 안녕하세요. 요즘 일하는 게 너무 힘듭니다. 부익부 빈익빈 현상을 어떻게 이해하고 어떤 마음가짐으로 살아야 할까요?

스님: 괴로움은 크게 두 종류인데, 첫째는 가
(EN/FR/中文-SUB)법륜스님 즉문즉설 깨달음과 죽음에 대해 알고 싶어요..ko.md
------------------------------------------------------------
질문자: 깨달음과 죽음에 대해 알고 싶습니다. 죽은 후에는 어떻게 되는지 궁금합니다.
스님: 죽어보면 알게 될 텐데 뭘 미리 물어요? 죽음이라는 건 살아있는 삶을 제대로 아는가에 


In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains import RetrievalQA

llm = ChatGoogleGenerativeAI(model="gemini-pro")

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    return_source_documents=True
)

# 질의 테스트
query = "법륜스님은 불안할 때 어떻게 하라고 하셨어?"
result = qa_chain.run(query)
print(result)
